In [97]:
from scipy.spatial import KDTree
import numpy as np
import random as rand

In [104]:
colors = ['red', 'blue', 'green', 'yellow', 'black']
weights = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
points = [(rand.randint(1, 100), rand.choice(colors), rand.choice(weights)) for _ in range(50)]

points_sorted = sorted(p for i, p in enumerate(points))
tree = KDTree(np.array([p[0] for p in points_sorted]).reshape(-1, 1))

[(73, 'yellow', 0.2), (80, 'yellow', 0.8), (36, 'yellow', 0.9), (94, 'blue', 0.8), (2, 'green', 0.8), (28, 'black', 0.3), (81, 'yellow', 0.4), (50, 'black', 0.3), (67, 'red', 0.5), (58, 'blue', 0.8), (38, 'yellow', 0.4), (95, 'blue', 0.6), (24, 'red', 0.7), (30, 'yellow', 0.5), (64, 'green', 0.9), (9, 'blue', 0.8), (50, 'green', 0.1), (29, 'green', 0.2), (1, 'green', 0.1), (3, 'red', 0.2), (14, 'green', 0.2), (71, 'green', 0.1), (42, 'green', 0.7), (32, 'green', 0.6), (73, 'blue', 0.3), (30, 'blue', 0.9), (20, 'green', 0.8), (31, 'blue', 0.9), (52, 'black', 0.4), (29, 'green', 0.4), (61, 'green', 0.4), (36, 'green', 0.2), (14, 'yellow', 0.9), (78, 'blue', 0.8), (100, 'blue', 0.5), (24, 'yellow', 0.2), (15, 'yellow', 0.9), (32, 'black', 0.4), (1, 'red', 0.4), (48, 'yellow', 0.1), (1, 'yellow', 0.6), (73, 'red', 0.4), (16, 'black', 0.7), (2, 'red', 0.8), (77, 'green', 0.5), (74, 'green', 0.2), (4, 'black', 0.3), (90, 'black', 0.8), (13, 'green', 0.2), (63, 'black', 0.4)]


In [105]:
query_range = (4, 20)

lower_bound_index = tree.query([query_range[0]], p=float('inf'))[1]
upper_bound_index = tree.query([query_range[1]], p=float('inf'))[1]

canonical_nodes = [p for p in points_sorted[lower_bound_index:upper_bound_index+1]]

print(canonical_nodes)

[(4, 'black', 0.3), (9, 'blue', 0.8), (13, 'green', 0.2), (14, 'green', 0.2), (14, 'yellow', 0.9), (15, 'yellow', 0.9), (16, 'black', 0.7), (20, 'green', 0.8)]


In [89]:
lower_bound_index

0

In [96]:
points_sorted